In [1]:
# import libraries
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

In [2]:
# preprocess images
def process_image(img):

    image = cv2.resize(img, (416, 416), interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
# draw detected boxes
def draw(image, boxes, scores):

    for box, score in zip(boxes, scores):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
                
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        
        cv2.putText(image, '{0} {1:.2f}'.format('person', score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)
        
        cv2.circle(image, (int((top + right) / 2), int((left + bottom) / 2)), radius=0, color=(0, 0, 255), thickness=10)

        print('class: {0}, score: {1:.2f}'.format('person', score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    array = [{'x': box[0], 'y': box[1], 'w': box[2], 'h': box[3]} for box in boxes]        

        # add cnn code to distinguish employee form customer
        # cropped = image[top:bottom, left:right]
        # cv2.imwrite('appearance/detected_'+ str(top) + '_' + str(left) + '_' + str(bottom) + '_' + str(right) +'.png', cropped) # only using the TOP parameter for encoding - later, use another professional way
    print(array)

In [4]:
# # calculate distance
# def distance(boxes):
#     person_location = {}

#     for i in range(0,len(boxes)):
#         x, y, w, h = boxes[i]

#         top = max(0, np.floor(x + 0.5).astype(int))
#         left = max(0, np.floor(y + 0.5).astype(int))
#         right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
#         bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

#         centerX = int((top + right) / 2)
#         centerY = int((left + bottom) / 2)

#         person_location["person" + "_" + str(i)] = centerX,centerY

#     return person_location

In [5]:
# # import cv2
# # import numpy as np

# # Load the camera matrix and distortion coefficients
# # with np.load('calib.npz') as X:
#     # mtx, dist = [X[i] for i in ('mtx', 'dist')]

# # Define image points and corresponding real-world points
# def homography(image):
#     image_points = np.array([
#         # [320, 240],
#         # [400, 240],
#         # [320, 320],
#         # [400, 320]

#         [0, 0],  # Example points
#         [image.shape[0], 0],
#         [0, image.shape[1]],
#         [image.shape[0], image.shape[1]]
#     ], dtype="float32")

#     world_points = np.array([
#         [0, 0],
#         [1, 0],
#         [0, 1],
#         [1, 1]
#     ], dtype="float32")

#     # Compute the homography matrix
#     H, status = cv2.findHomography(image_points, world_points)

# # Function to transform points using the homography matrix
# def warp_point(point, H):
#     point = np.array([point[0], point[1], 1.0]).reshape((3, 1))
#     warped_point = np.dot(H, point)
#     warped_point = warped_point / warped_point[2]
#     return (warped_point[0], warped_point[1])

# # Example points (centers of bounding boxes)
# points_to_transform = [
#         (340, 260),
#         (360, 260),
#         # Add more points as needed
#     ]

# # Transform the points
# transformed_points = [warp_point(pt, H) for pt in points_to_transform]

# # Calculate the distance between transformed points
# def calculate_euclidean_distance(point1, point2):
#     x1, y1 = point1
#     x2, y2 = point2
#     return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# for i in range(len(transformed_points)):
#     for j in range(i + 1, len(transformed_points)):
#         dist = calculate_distance(transformed_points[i], transformed_points[j])
#         print(f"Distance between point {i} and point {j}: {dist:.2f} units")


In [4]:
# detect images
def detect_image(image, yolo):
    
    processed_image = process_image(image)

    start = time.time()
    boxes, _, scores = yolo.predict(processed_image, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

#   explain how print statement works
#   {0} indicates that a value will be inserted at this position
#   `:.2f` is a format specifier that tells Python to format the value as a floating-point number with 2 decimal places
#   `.format(end - start)` This method is used to insert values into the placeholders within a string
    
    if boxes is not None:
        # person_location = distance(boxes)
        # print(person_location)
        draw(image, boxes, scores)

    return image

In [5]:
# detect videos
def detect_video(video, yolo):
    
    # use yolo v3 to detect video.
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    

In [6]:
# import yolo model
yolo = YOLO(obj_threshold=0.5, nms_threshold=0.5)

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [7]:
# test images
for i in range(0,9):
    path = 'data/dataset/'+ str(i) +'.png'
    print(path)
    image = cv2.imread(path)
    print(image.shape)
    image = detect_image(image, yolo)
    cv2.imwrite('output/detected_'+ str(i) +'.png', image)


data/dataset/0.png
(548, 543, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
time: 9.36s
class: person, score: 1.00
box coordinate x,y,w,h: [ 18.4972295   39.65744426 157.71224694 323.0362184 ]
[{'x': 18.497229496599957, 'y': 39.65744426184179, 'w': 157.71224694359964, 'h': 323.0362184027209}]
data/dataset/1.png
(569, 640, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
time: 1.25s
class: person, score: 0.95
box coordinate x,y,w,h: [187.41739003  50.26758903 241.21990485 490.59628982]
[{'x': 187.41739002830462, 'y': 50.267589029675904, 'w': 241.21990484864327, 'h': 490.59628981990096}]
data/dataset/2.png
(567, 573, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 863ms/step
time: 0.97s
class: person, score: 0.96
box coordinate x,y,w,h: [300.85994719  46.88698708 218.51947312 286.23123276]
[{'x': 300.85994719420376, 'y': 46.886987078243656, 'w': 218.5194731202455, 'h': 286.2312327645248}]
data/dataset/3.png
(535, 535, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 643ms/step
time: 0.73s
class: person, score: 0.99
box coordinate x,y,w,h:

In [ ]:
# # test video
# for i in range(0,9):
#     path = 't.mp4'
#     # image = cv2.imread(path)
#     detect_video(path, yolo)
#     # cv2.imwrite('output/detected_'+ str(i) +'.png', image)


1/1 [==============================] - 2s 2s/step
time: 2.42s
1/1 [==============================] - 0s 465ms/step
time: 0.59s
1/1 [==============================] - 1s 726ms/step
time: 0.87s
1/1 [==============================] - 0s 478ms/step
time: 0.55s
1/1 [==============================] - 0s 432ms/step
time: 0.50s
1/1 [==============================] - 0s 438ms/step
time: 0.50s
1/1 [==============================] - 1s 531ms/step
time: 0.63s
1/1 [==============================] - 1s 585ms/step
time: 0.65s
1/1 [==============================] - 1s 513ms/step
time: 0.63s
1/1 [==============================] - 0s 475ms/step
time: 0.55s
1/1 [==============================] - 0s 461ms/step
time: 0.51s
1/1 [==============================] - 1s 520ms/step
time: 0.60s
1/1 [==============================] - 1s 524ms/step
time: 0.58s
1/1 [==============================] - 1s 561ms/step
time: 0.65s
1/1 [==============================] - 0s 397ms/step
time: 0.51s
1/1 [=======================

KeyboardInterrupt: 